# Multinomial Chebyshev Approximations in Python
## Advanced Quantitative Marketing - Arjun Gopinath

In this Jupyter notebook, I display all relevant results from the approximations assignment.

In [1]:
# Frequently used packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import gumbel_r
from scipy.special import logsumexp
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib
from numpy.polynomial import chebyshev as ch
from itertools import product

# Imports for displaying tables in Jupyter notebooks
from IPython.display import display, HTML
import tabulate

# Frequently used commands
inv, ax, det = np.linalg.inv, np.newaxis, np.linalg.det
cos, pi, arccos, log = np.cos, np.pi, np.arccos, np.log

# Random number generator
unif = np.random.uniform

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] =  r'\usepackage{amssymb}' + r'\usepackage{amsmath}' + r'\usepackage{xcolor}' + r'\renewcommand*\familydefault{\sfdefault}'
matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams['pgf.preamble'] = r'\usepackage[utf8x]{inputenc}' + r'\usepackage{amssymb}' + r'\usepackage[T1]{fontenc}' + r'\usepackage{amsmath}' + r'\usepackage{sansmath}'

## Definition of the `ChebyshevApproximator` Class

In [2]:
class ChebyshevApproximator:
    
    """
        Class used to compute Chebyshev interpolations for multidimensional functions.
    """
    
    def __init__(self,
                 f: 'Function to be approximated, function',
                 D: "The dimension of the rectangle, integer",
                 N: "The degree of Chebyshev approximation, integer", 
                 M: 'The number of nodes in the approximation algorithm, integer', 
                 bounds: 'Bounds of the rectangle for the approximation, np.array of dimension D x 2'):
        
        # Defining characteristics for the object used to perform Chebyshev approximation.
        self.f, self.D, self.N, self.M, self.bounds = f, D, N, M, bounds

    def initializeChebyshevApproximator(self):
        
        """
            Initialization of the Chebyshev routine.
        """

        D, N, M = self.D, self.N, self.M

        # Computation of nodes along a single dimension.
        self.nodes1D = -cos(pi * (np.arange(1, M + 1) - 0.5)/M) 

        # D-fold cartesian product of Chebyshev nodes.
        self.X = np.array(list(product(self.nodes1D, repeat=D)))

        # Computing the tensor product basis polynomial matrix at each D-dim Chebyshev node.
        self.T = np.apply_along_axis(lambda x : self.calculateChebyshevPolynomials(val=x), 1, self.X)


    def calculateChebyshevPolynomials(self, 
                                      val: 'Slice at which the polynomial is evaluated, of size D'):

        """
            Method used for degree-N Chebyshev polynomial computation at a dimension-D point.
        """

        # Initial value of the recursive Chebyshev polynomial.
        ChebySlice = 1

        # Apply Kronecker product of computed Chebyshev polynomials over each dimension.
        for d in range(self.D): 
            ChebySlice = np.kron(cos(np.arange(0, self.N + 1)[ax, :] * arccos(val[d]))[ax, :], ChebySlice)
        
        return (np.squeeze(ChebySlice))


    def calculateChebyshevCoefficients(self):

        """
            Method used to regress function evaluated at nodes on Chebyshev polynomials.
        """

        f, X, T = self.f, self.X, self.T
        a, b = self.bounds[:, 0][ax, :], self.bounds[:, 1][ax, :]

        # Converting the Chebyshev nodes to the rectangle provided in the input.
        Z = 0.5 * (X + 1) * (b - a) + a
        
        # Evaluating the function at the adjusted Chebyshev nodes
        Y = f(Z)[:, ax]
        
        # Using the LS result to extract the Chebyshev coefficients
        self.coeffChebyshev = np.diag(np.diag(T.T @ T) ** -1) @ T.T @ Y
        
    def evaluateChebyshev(self, 
                          values: 'A vector or matrix with which to evaluate the Chebyshev approximation.'):

        """
            Run the Chebyshev approximation routine for a given set of values.
        """

        a, b = self.bounds[:, 0][ax, :], self.bounds[:, 1][ax, :]

        # Convert values from the function domain to the [-1, 1]^D rectangle.
        nu = 2 * (values - a)/(b - a) - 1

        # Apply the tensor Chebyshev basis polynomials to the transformed input values.
        T = np.apply_along_axis(lambda x : self.calculateChebyshevPolynomials(val=x), 1, nu)
        
        # Evaluate the function using the Chebyshev coefficients associated with the Chebyshev basis polynomials.
        y_out = T @ self.coeffChebyshev

        return np.squeeze(y_out)

                
        

## Computing the Approximation Error

In [3]:
def deriveChebyshevApproxError(N: 'Chebyshev polynomial degree',
                               M: 'Number of Chebyshev nodes',
                               func: 'Function to be approximated',
                               rect_bounds: 'Bounds of the rectangle for evaluation',
                               values: 'Values at which the function is approximated'):

    """
        A user-defined function that assesses the performance of the Chebyshev approximation for 
        a function over a given rectangle in R^D space using a specified number of nodes and polynomial degree.
    """

    dim, _ = rect_bounds.shape
    
    # Initializing an object of the ChebyshevApproximator class.
    approx_obj = ChebyshevApproximator(f=func, D=dim, N=N, M=M, bounds=rect_bounds)
    approx_obj.initializeChebyshevApproximator()

    # Computing the Chebyshev coefficients associated with the provided function.
    approx_obj.calculateChebyshevCoefficients()

    diff = np.abs(approx_obj.evaluateChebyshev(values=values) - func(random_values))

    return np.array([diff.mean(), diff.max()])


## Setting up the test for the approximator

In [4]:
# Definitions of the test functions.

def func_1(X):

    x, y, z = X[:, 0], X[:, 1], X[:, 2]

    return x * (z ** 3) + y * z + (x ** 2) * y * (z ** 2)

def func_2(X):

    x, y, z = X[:, 0], X[:, 1], X[:, 2]

    return x * log(5 + y) * z

def func_3(X):

    x, y, z = X[:, 0], X[:, 1], X[:, 2]

    return (x ** 2) * cos(y) * np.exp(z)

# Bounds of the rectangle over which each function is approximated.
rect_bounds = np.array([[-5, 2], [-2, 4], [-3, 3]])

# Setting up the space over which the Chebyshev approximations will be computed via vectorization.
N_vec, M_vec, func_vec = np.arange(3, 9), np.arange(7, 12), [func_1, func_2, func_3]
approx_space = np.array([(N, M, func) for func in func_vec for N in N_vec for M in M_vec if M > N + 1])

In [5]:
# Drawing random uniform draws from the specified rectangle.

N_draws = 10000

dim, _ = rect_bounds.shape

random_values = np.array([])

for d in range(dim):

    random_values = np.append(random_values,
                              unif(low=rect_bounds[d, 0], 
                                   high=rect_bounds[d, 1], 
                                   size=N_draws))


random_values = random_values.reshape([dim, N_draws]).T


## Vectorized Calculations of the Chebyshev Approximation Errors

In [6]:
diff_res = np.apply_along_axis(
    lambda X : deriveChebyshevApproxError(
        N=X[0], M=X[1], func=X[2], rect_bounds=rect_bounds, values=random_values
        ), 1, approx_space)

In [9]:
# Converting the results into a Pandas dataframe.

columns_df = ['Degree', '# Nodes', 'Function', 'Mean Diff.', 'Max Diff.']

results_df = pd.DataFrame(np.append(approx_space, diff_res, axis=1), 
                          columns=columns_df)

func_map = {func_1 : '$f(x)$', func_2 : '$g(x)$', func_3: '$h(x)$'}
results_df.replace({'Function': func_map}, inplace=True)

results_df.set_index(['Function', 'Degree', '# Nodes'], inplace=True)

pd.set_option('display.float_format', lambda x: '%.3e' % x if x < 0.001 else '%.3f' % x)
pd.set_option('display.max_rows', None)

In [10]:
results_df

Mean Diff. Max Diff.
Function Degree # Nodes                     
$f(x)$   3      7        1.469e-13 9.095e-13
                8        1.385e-13 9.095e-13
                9        6.679e-14 9.095e-13
                10       9.752e-14 6.253e-13
                11       6.338e-14 5.684e-13
         4      7        1.476e-13 9.095e-13
                8        1.453e-13 7.958e-13
                9        7.391e-14 1.023e-12
                10       1.014e-13 7.958e-13
                11       7.230e-14 9.095e-13
         5      7        1.724e-13 1.307e-12
                8        1.535e-13 1.194e-12
                9        7.891e-14 1.137e-12
                10       1.297e-13 1.478e-12
                11       7.764e-14 9.095e-13
         6      8        1.630e-13 1.023e-12
                9        1.066e-13 1.478e-12
                10       1.318e-13 1.307e-12
                11       1.078e-13 1.023e-12
         7      9        1.168e-13 2.160e-12
                10       1.395e-13 1.933e-12
                11       1.233e-13 1.307e-12
         8      10       1.615e-13 2.728e-12
                11       1.411e-13 2.274e-12
$g(x)$   3      7            0.005     0.043
                8            0.005     0.043
                9            0.005     0.043
                10           0.005     0.043
                11           0.005     0.043
         4      7            0.001     0.009
                8            0.001     0.009
                9            0.001     0.009
                10           0.001     0.009
                11           0.001     0.009
         5      7        2.360e-04     0.002
                8        2.366e-04     0.002
                9        2.367e-04     0.002
                10       2.367e-04     0.002
                11       2.367e-04     0.002
         6      8        5.381e-05 4.973e-04
                9        5.398e-05 4.921e-04
                10       5.400e-05 4.918e-04
                11       5.400e-05 4.918e-04
         7      9        1.260e-05 1.081e-04
                10       1.264e-05 1.071e-04
                11       1.264e-05 1.071e-04
         8      10       3.019e-06 2.626e-05
                11       3.028e-06 2.594e-05
$h(x)$   3      7            2.846    81.991
                8            2.846     81.99
                9            2.846     81.99
                10           2.846     81.99
                11           2.846     81.99
         4      7            1.169    31.068
                8            1.169    31.069
                9            1.169    31.069
                10           1.169    31.069
                11           1.169    31.069
         5      7            0.226     6.751
                8            0.226     6.772
                9            0.226     6.771
                10           0.226     6.771
                11           0.226     6.771
         6      8            0.066     2.063
                9            0.066     2.066
                10           0.066     2.066
                11           0.066     2.066
         7      9            0.009     0.279
                10           0.009     0.279
                11           0.009     0.279
         8      10           0.002     0.067
                11           0.002     0.067

## Discussion

From the results above, we see that the type of function approximated affects the quality of the operation. The polynomial function $f(x)$ is well approximated with errors running into the order of $10^{-12}$, whereas approximating $g(x)$ and $h(x)$ yields comparatively higher errors holding the degree and number of nodes fixed. Non-polynomial functions can be better approximated with higher number of polynomial basis functions. Furthermore, increasing the number of nodes when holding the degree fixed doesn't improve the approximation by much. However, increasing the degree of the Chebyshev polynomials used has a noticeable improvement on the approximation given a fixed number of nodes. Clearly, as $N \to \infty$, the approximation will perfectly replicate the function.